In [1]:
import pandas as pd
import numpy as np
import gzip
import json
import shutil
from pathlib import Path

The aim is to read the data from a CSV file after wrangling it, and then read it with csv.DictReader and append it to the JSON file we created.

# processing data in csv file

The overall purpose of this wrangle() function is to read a CSV file, perform various data cleaning and transformation steps, and return the resulting DataFrame with the modifications applied.

In [2]:
def wrangle(file_path):
    df = pd.read_csv(file_path,low_memory=False)
    
    start_index = 1  # Starting index for the new column names
    column_range = range(0, 64)  # Range of columns to be renamed

    for i, col_index in enumerate(column_range):
        new_col_name = f"Attr_{start_index + i}"
        df.rename(columns={df.columns[col_index]: new_col_name}, inplace=True)
    
    # insert column 'company_id'
    df.insert(0, 'company_id', range(1, len(df) + 1))

    # rename 'class' column to 'bankrupt'
    df = df.rename(columns={'class':'bankrupt'})
    
    # remove '?' values 
    df = df.replace({'?': 'NaN'})
    
    df= df.astype({c: "float64" for c in df.columns[1:65]})
    df['bankrupt'] = df['bankrupt'].astype(bool)
    
    return df

In [27]:
df = wrangle(r"E:\WorldQuant\project_5\data\Poland_bankruptcy.csv")
print(df.shape)
df.head()

(10503, 66)


,company_id,Attr_1,Attr_2,Attr_3,Attr_4,Attr_5,Attr_6,Attr_7,Attr_8,Attr_9,...,Attr_56,Attr_57,Attr_58,Attr_59,Attr_60,Attr_61,Attr_62,Attr_63,Attr_64,bankrupt
0,1,0.174190,0.41299,0.14371,1.3480,-28.9820,0.60383,0.219460,1.1225,1.1961,...,0.163960,0.375740,0.83604,0.000007,9.7145,6.2813,84.291,4.3303,4.0341,False
1,2,0.146240,0.46038,0.28230,1.6294,2.5952,0.00000,0.171850,1.1721,1.6018,...,0.027516,0.271000,0.90108,0.000000,5.9882,4.1103,102.190,3.5716,5.9500,False
2,3,0.000595,0.22612,0.48839,3.1599,84.8740,0.19114,0.004572,2.9881,1.0077,...,0.007639,0.000881,0.99236,0.000000,6.7742,3.7922,64.846,5.6287,4.4581,False
3,4,0.024526,0.43236,0.27546,1.7833,-10.1050,0.56944,0.024526,1.3057,1.0509,...,0.048398,0.043445,0.95160,0.142980,4.2286,5.0528,98.783,3.6950,3.4844,False
4,5,0.188290,0.41504,0.34231,1.9279,-58.2740,0.00000,0.233580,1.4094,1.3393,...,0.176480,0.321880,0.82635,0.073039,2.5912,7.0756,100.540,3.6303,4.6375,False


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10503 entries, 0 to 10502
Data columns (total 66 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   company_id  10503 non-null  int64  
 1   Attr_1      10503 non-null  float64
 2   Attr_2      10503 non-null  float64
 3   Attr_3      10503 non-null  float64
 4   Attr_4      10485 non-null  float64
 5   Attr_5      10478 non-null  float64
 6   Attr_6      10503 non-null  float64
 7   Attr_7      10503 non-null  float64
 8   Attr_8      10489 non-null  float64
 9   Attr_9      10500 non-null  float64
 10  Attr_10     10503 non-null  float64
 11  Attr_11     10503 non-null  float64
 12  Attr_12     10485 non-null  float64
 13  Attr_13     10460 non-null  float64
 14  Attr_14     10503 non-null  float64
 15  Attr_15     10495 non-null  float64
 16  Attr_16     10489 non-null  float64
 17  Attr_17     10489 non-null  float64
 18  Attr_18     10503 non-null  float64
 19  Attr_19     10460 non-nul

### Save changes we made on csv file by wrangle function to new csv file `Poland_bankruptcy_modified.csv`

In [5]:
df.to_csv(r"E:\WorldQuant\project_5\data\Poland_bankruptcy_modified.csv", index=False)

###### After procession  the CSV file,we extracts random rows, sorts them based on the first column, saves them to a new CSV file, and removes the extracted rows from the original file.

In [6]:
import csv
import random

input_file = r'E:\WorldQuant\project_5\data\Poland_bankruptcy_modified.csv'
output_file = r'E:\WorldQuant\project_5\data\Poland_bankruptcy_modified_test.csv'

# Step 1: Read the CSV file
with open(input_file, 'r') as file:
    reader = csv.reader(file)
    rows = list(reader)

    header_row = rows[0]
    data_rows = rows[1:]

    num_data_rows = len(data_rows)
    num_random_rows = 526  # Define the number of random rows you want to extract
    random_indices = random.sample(range(num_data_rows), num_random_rows)

    random_rows = [data_rows[index] for index in random_indices]

    # Step 2: Sort the extracted rows based on the first column
    sorted_rows = sorted(random_rows, key=lambda row: float(row[0]))

    # Step 3: Save the sorted extracted rows in a new CSV file with the header
    with open(output_file, 'w', newline='') as output:
        writer = csv.writer(output)
        writer.writerow(header_row)  # Write the header row
        writer.writerows(sorted_rows)

    # Step 4: Remove the extracted rows from the original CSV file
    remaining_rows = [row for index, row in enumerate(data_rows) if index not in random_indices]

    with open(input_file, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(header_row)  # Write the header row
        writer.writerows(remaining_rows)

In [26]:
original_csv = pd.read_csv(r'E:\WorldQuant\project_5\data\Poland_bankruptcy_modified.csv')
print(original_csv.shape)
original_csv.head()

(9977, 66)


,company_id,Attr_1,Attr_2,Attr_3,Attr_4,Attr_5,Attr_6,Attr_7,Attr_8,Attr_9,...,Attr_56,Attr_57,Attr_58,Attr_59,Attr_60,Attr_61,Attr_62,Attr_63,Attr_64,bankrupt
0,1,0.174190,0.41299,0.14371,1.3480,-28.9820,0.60383,0.219460,1.1225,1.1961,...,0.163960,0.375740,0.83604,0.000007,9.7145,6.2813,84.291,4.3303,4.0341,False
1,2,0.146240,0.46038,0.28230,1.6294,2.5952,0.00000,0.171850,1.1721,1.6018,...,0.027516,0.271000,0.90108,0.000000,5.9882,4.1103,102.190,3.5716,5.9500,False
2,3,0.000595,0.22612,0.48839,3.1599,84.8740,0.19114,0.004572,2.9881,1.0077,...,0.007639,0.000881,0.99236,0.000000,6.7742,3.7922,64.846,5.6287,4.4581,False
3,4,0.024526,0.43236,0.27546,1.7833,-10.1050,0.56944,0.024526,1.3057,1.0509,...,0.048398,0.043445,0.95160,0.142980,4.2286,5.0528,98.783,3.6950,3.4844,False
4,5,0.188290,0.41504,0.34231,1.9279,-58.2740,0.00000,0.233580,1.4094,1.3393,...,0.176480,0.321880,0.82635,0.073039,2.5912,7.0756,100.540,3.6303,4.6375,False


In [8]:
import csv
filename = r'E:\WorldQuant\project_5\data\Poland_bankruptcy_modified_test.csv'
column_index = 65

with open(filename, 'r', newline='') as file:
    reader = csv.reader(file)
    rows = [row[:column_index] + row[column_index+1:] for row in reader]

with open(filename, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(rows)

In [25]:
extracted_csv = pd.read_csv(r'E:\WorldQuant\project_5\data\Poland_bankruptcy_modified_test.csv')
print(extracted_csv.shape)
extracted_csv.head()

(526, 65)


,company_id,Attr_1,Attr_2,Attr_3,Attr_4,Attr_5,Attr_6,Attr_7,Attr_8,Attr_9,...,Attr_55,Attr_56,Attr_57,Attr_58,Attr_59,Attr_60,Attr_61,Attr_62,Attr_63,Attr_64
0,9,0.238950,0.554730,0.40697,1.76090,-22.9070,0.00000,0.297140,0.80268,2.33170,...,3315.200,0.140380,0.536630,0.87292,0.044584,4.4527,7.1847,83.727,4.3594,40.09700
1,28,0.150790,0.584200,0.23091,1.44850,85.8890,0.00000,0.185430,0.71174,0.87949,...,6854.900,0.215930,0.362660,0.79215,0.000000,12.0940,1.3080,213.660,1.7083,3.45900
2,40,0.013141,0.386260,0.38047,2.24300,44.2000,0.00000,0.013141,1.58890,0.88321,...,3220.000,-0.132720,0.021412,0.98720,0.000000,3.3917,4.1778,126.500,2.8855,2.81780
3,81,-0.010051,0.545700,-0.00150,0.99725,-31.1180,0.12076,-0.010051,0.83252,3.35550,...,-2.081,-0.008543,-0.022123,1.00300,0.000000,11.8690,13.5820,59.359,6.1490,7.36170
4,83,0.033372,0.089748,0.10903,2.27890,-2.6961,0.28508,0.033372,10.14200,0.28484,...,2097.100,-0.318780,0.036663,0.92596,0.000000,2.5771,4.0406,109.240,3.3413,0.35351


### Create JSON file 

In [10]:
Poland_bankruptcy_modified = {
     "schema": {
               "fields":[{'name': 'company_id', 'type': 'integer'},
                         {'name': 'Attr_1', 'type': 'number'},
                         {'name': 'Attr_2', 'type': 'number'},
                         {'name': 'Attr_3', 'type': 'number'},
                         {'name': 'Attr_4', 'type': 'number'},
                         {'name': 'Attr_5', 'type': 'number'},
                         {'name': 'Attr_6', 'type': 'number'},
                         {'name': 'Attr_7', 'type': 'number'},
                         {'name': 'Attr_8', 'type': 'number'},
                         {'name': 'Attr_9', 'type': 'number'},
                         {'name': 'Attr_10', 'type': 'number'},
                         {'name': 'Attr_11', 'type': 'number'},
                         {'name': 'Attr_12', 'type': 'number'},
                         {'name': 'Attr_13', 'type': 'number'},
                         {'name': 'Attr_14', 'type': 'number'},
                         {'name': 'Attr_15', 'type': 'number'},
                         {'name': 'Attr_16', 'type': 'number'},
                         {'name': 'Attr_17', 'type': 'number'},
                         {'name': 'Attr_18', 'type': 'number'},
                         {'name': 'Attr_19', 'type': 'number'},
                         {'name': 'Attr_20', 'type': 'number'},
                         {'name': 'Attr_21', 'type': 'number'},
                         {'name': 'Attr_22', 'type': 'number'},
                         {'name': 'Attr_23', 'type': 'number'},
                         {'name': 'Attr_24', 'type': 'number'},
                         {'name': 'Attr_25', 'type': 'number'},
                         {'name': 'Attr_26', 'type': 'number'},
                         {'name': 'Attr_27', 'type': 'number'},
                         {'name': 'Attr_28', 'type': 'number'},
                         {'name': 'Attr_29', 'type': 'number'},
                         {'name': 'Attr_30', 'type': 'number'},
                         {'name': 'Attr_31', 'type': 'number'},
                         {'name': 'Attr_32', 'type': 'number'},
                         {'name': 'Attr_33', 'type': 'number'},
                         {'name': 'Attr_34', 'type': 'number'},
                         {'name': 'Attr_35', 'type': 'number'},
                         {'name': 'Attr_36', 'type': 'number'},
                         {'name': 'Attr_37', 'type': 'number'},
                         {'name': 'Attr_38', 'type': 'number'},
                         {'name': 'Attr_39', 'type': 'number'},
                         {'name': 'Attr_40', 'type': 'number'},
                         {'name': 'Attr_41', 'type': 'number'},
                         {'name': 'Attr_42', 'type': 'number'},
                         {'name': 'Attr_43', 'type': 'number'},
                         {'name': 'Attr_44', 'type': 'number'},
                         {'name': 'Attr_45', 'type': 'number'},
                         {'name': 'Attr_46', 'type': 'number'},
                         {'name': 'Attr_47', 'type': 'number'},
                         {'name': 'Attr_48', 'type': 'number'},
                         {'name': 'Attr_49', 'type': 'number'},
                         {'name': 'Attr_50', 'type': 'number'},
                         {'name': 'Attr_51', 'type': 'number'},
                         {'name': 'Attr_52', 'type': 'number'},
                         {'name': 'Attr_53', 'type': 'number'},
                         {'name': 'Attr_54', 'type': 'number'},
                         {'name': 'Attr_55', 'type': 'number'},
                         {'name': 'Attr_56', 'type': 'number'},
                         {'name': 'Attr_57', 'type': 'number'},
                         {'name': 'Attr_58', 'type': 'number'},
                         {'name': 'Attr_59', 'type': 'number'},
                         {'name': 'Attr_60', 'type': 'number'},
                         {'name': 'Attr_61', 'type': 'number'},
                         {'name': 'Attr_62', 'type': 'number'},
                         {'name': 'Attr_63', 'type': 'number'},
                         {'name': 'Attr_64', 'type': 'number'},
                         {'name': 'bankrupt', 'type': 'boolean'}],
               "primary_key":["company_id"],
                "pandas_version":"0.20.0"
              },
    "data": [],
    "metadata": {
                 "title":"Ensemble Boosted Trees with Synthetic Features Generation in Application to Bankruptcy Prediction",
                 "authors":"Zieba, M., Tomczak, S. K., & Tomczak, J. M.",
                 "journal":"Expert Systems with Applications",
                 "publicationYear":2016,
                 "dataYear":2009,
                 "articleLink":"doi:10.1016/j.eswa.2016.04.001",
                 "datasetLink":"https://archive.ics.uci.edu/ml/datasets/Polish+companies+bankruptcy+data"
                }
    }
with open(r"E:\WorldQuant\project_5\data\Poland_bankruptcy_modified.json","w") as file:
    json.dump(Poland_bankruptcy_modified,file,ensure_ascii=False,indent=4)

# Note

The csv.`DictReader` function in Python's csv module reads values from a CSV file as strings by default. This behavior is intentional because CSV files are text-based and do not have explicit data types defined for the values. Therefore, all values in the CSV file are initially treated as strings.

So if we append the data from a CSV file as dictionaries, the data types of values will be treated as strings. To avoid this problem, there are two solutions: one is to fix the data types of values before appending them to the JSON file, and the other is to fix the data types of values after appending them to the JSON file.

##### In this Notebook, we will fix the data types of values before appending them to the JSON file.

By the end of the loop, the `dict_data_1` list will contain dictionaries, with each dictionary representing a row of data from the CSV file.

In [11]:
dict_data_1=[]
import csv
        
with open(r"E:\WorldQuant\project_5\data\Poland_bankruptcy_modified.csv", newline='') as f:
    reader = csv.DictReader(f)
    
    for row in reader:
        
        myfixedrow = {k: (None if v=="" else (True if v=="True" else (False if v=="False" else (int(v)if k=="company_id" else float(v))))) for k,v in row.items()}
        
        dict_data_1.append(myfixedrow)

In [12]:

json_file = Path(r"E:\WorldQuant\project_5\data\Poland_bankruptcy_modified.json")
json_data = json.loads(json_file.read_text())

data = [index for index in dict_data_1]
json_data["data"] = data

json_file.write_text(json.dumps(json_data))

12901871

In [13]:
def wrangle_1(filename):
    
    # Open compressed file, load into dictionary
    with open(filename,"r") as f:
        data = json.load(f)
    # Load dictionary into DataFrame, set index
    df=pd.DataFrame.from_dict(data["data"])
    
    return df

In [28]:
df1 = wrangle_1(r"E:\WorldQuant\project_5\data\Poland_bankruptcy_modified.json")
print(df1.shape)
df1.head()

(9977, 66)


,company_id,Attr_1,Attr_2,Attr_3,Attr_4,Attr_5,Attr_6,Attr_7,Attr_8,Attr_9,...,Attr_56,Attr_57,Attr_58,Attr_59,Attr_60,Attr_61,Attr_62,Attr_63,Attr_64,bankrupt
0,1,0.174190,0.41299,0.14371,1.3480,-28.9820,0.60383,0.219460,1.1225,1.1961,...,0.163960,0.375740,0.83604,0.000007,9.7145,6.2813,84.291,4.3303,4.0341,False
1,2,0.146240,0.46038,0.28230,1.6294,2.5952,0.00000,0.171850,1.1721,1.6018,...,0.027516,0.271000,0.90108,0.000000,5.9882,4.1103,102.190,3.5716,5.9500,False
2,3,0.000595,0.22612,0.48839,3.1599,84.8740,0.19114,0.004572,2.9881,1.0077,...,0.007639,0.000881,0.99236,0.000000,6.7742,3.7922,64.846,5.6287,4.4581,False
3,4,0.024526,0.43236,0.27546,1.7833,-10.1050,0.56944,0.024526,1.3057,1.0509,...,0.048398,0.043445,0.95160,0.142980,4.2286,5.0528,98.783,3.6950,3.4844,False
4,5,0.188290,0.41504,0.34231,1.9279,-58.2740,0.00000,0.233580,1.4094,1.3393,...,0.176480,0.321880,0.82635,0.073039,2.5912,7.0756,100.540,3.6303,4.6375,False


In [15]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9977 entries, 0 to 9976
Data columns (total 66 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   company_id  9977 non-null   int64  
 1   Attr_1      9977 non-null   float64
 2   Attr_2      9977 non-null   float64
 3   Attr_3      9977 non-null   float64
 4   Attr_4      9959 non-null   float64
 5   Attr_5      9955 non-null   float64
 6   Attr_6      9977 non-null   float64
 7   Attr_7      9977 non-null   float64
 8   Attr_8      9963 non-null   float64
 9   Attr_9      9974 non-null   float64
 10  Attr_10     9977 non-null   float64
 11  Attr_11     9977 non-null   float64
 12  Attr_12     9959 non-null   float64
 13  Attr_13     9936 non-null   float64
 14  Attr_14     9977 non-null   float64
 15  Attr_15     9969 non-null   float64
 16  Attr_16     9963 non-null   float64
 17  Attr_17     9963 non-null   float64
 18  Attr_18     9977 non-null   float64
 19  Attr_19     9936 non-null  

In [16]:
# gzip compress file
with open(r"E:\WorldQuant\project_5\data\Poland_bankruptcy_modified.json",'rb') as f_input:
    with gzip.open(r"E:\WorldQuant\project_5\data\Poland_bankruptcy_modified.json.gz",'wb') as f_output:
        shutil.copyfileobj(f_input,f_output)


In [17]:
Poland_bankruptcy_modified_test = {
     "schema": {
               "fields":[{'name': 'company_id', 'type': 'integer'},
                         {'name': 'Attr_1', 'type': 'number'},
                         {'name': 'Attr_2', 'type': 'number'},
                         {'name': 'Attr_3', 'type': 'number'},
                         {'name': 'Attr_4', 'type': 'number'},
                         {'name': 'Attr_5', 'type': 'number'},
                         {'name': 'Attr_6', 'type': 'number'},
                         {'name': 'Attr_7', 'type': 'number'},
                         {'name': 'Attr_8', 'type': 'number'},
                         {'name': 'Attr_9', 'type': 'number'},
                         {'name': 'Attr_10', 'type': 'number'},
                         {'name': 'Attr_11', 'type': 'number'},
                         {'name': 'Attr_12', 'type': 'number'},
                         {'name': 'Attr_13', 'type': 'number'},
                         {'name': 'Attr_14', 'type': 'number'},
                         {'name': 'Attr_15', 'type': 'number'},
                         {'name': 'Attr_16', 'type': 'number'},
                         {'name': 'Attr_17', 'type': 'number'},
                         {'name': 'Attr_18', 'type': 'number'},
                         {'name': 'Attr_19', 'type': 'number'},
                         {'name': 'Attr_20', 'type': 'number'},
                         {'name': 'Attr_21', 'type': 'number'},
                         {'name': 'Attr_22', 'type': 'number'},
                         {'name': 'Attr_23', 'type': 'number'},
                         {'name': 'Attr_24', 'type': 'number'},
                         {'name': 'Attr_25', 'type': 'number'},
                         {'name': 'Attr_26', 'type': 'number'},
                         {'name': 'Attr_27', 'type': 'number'},
                         {'name': 'Attr_28', 'type': 'number'},
                         {'name': 'Attr_29', 'type': 'number'},
                         {'name': 'Attr_30', 'type': 'number'},
                         {'name': 'Attr_31', 'type': 'number'},
                         {'name': 'Attr_32', 'type': 'number'},
                         {'name': 'Attr_33', 'type': 'number'},
                         {'name': 'Attr_34', 'type': 'number'},
                         {'name': 'Attr_35', 'type': 'number'},
                         {'name': 'Attr_36', 'type': 'number'},
                         {'name': 'Attr_37', 'type': 'number'},
                         {'name': 'Attr_38', 'type': 'number'},
                         {'name': 'Attr_39', 'type': 'number'},
                         {'name': 'Attr_40', 'type': 'number'},
                         {'name': 'Attr_41', 'type': 'number'},
                         {'name': 'Attr_42', 'type': 'number'},
                         {'name': 'Attr_43', 'type': 'number'},
                         {'name': 'Attr_44', 'type': 'number'},
                         {'name': 'Attr_45', 'type': 'number'},
                         {'name': 'Attr_46', 'type': 'number'},
                         {'name': 'Attr_47', 'type': 'number'},
                         {'name': 'Attr_48', 'type': 'number'},
                         {'name': 'Attr_49', 'type': 'number'},
                         {'name': 'Attr_50', 'type': 'number'},
                         {'name': 'Attr_51', 'type': 'number'},
                         {'name': 'Attr_52', 'type': 'number'},
                         {'name': 'Attr_53', 'type': 'number'},
                         {'name': 'Attr_54', 'type': 'number'},
                         {'name': 'Attr_55', 'type': 'number'},
                         {'name': 'Attr_56', 'type': 'number'},
                         {'name': 'Attr_57', 'type': 'number'},
                         {'name': 'Attr_58', 'type': 'number'},
                         {'name': 'Attr_59', 'type': 'number'},
                         {'name': 'Attr_60', 'type': 'number'},
                         {'name': 'Attr_61', 'type': 'number'},
                         {'name': 'Attr_62', 'type': 'number'},
                         {'name': 'Attr_63', 'type': 'number'},
                         {'name': 'Attr_64', 'type': 'number'}],
               "primary_key":["company_id"],
                "pandas_version":"0.20.0"
              },
    "data": [],
    "metadata": {
                 "title":"Ensemble Boosted Trees with Synthetic Features Generation in Application to Bankruptcy Prediction",
                 "authors":"Zieba, M., Tomczak, S. K., & Tomczak, J. M.",
                 "journal":"Expert Systems with Applications",
                 "publicationYear":2016,
                 "dataYear":2009,
                 "articleLink":"doi:10.1016/j.eswa.2016.04.001",
                 "datasetLink":"https://archive.ics.uci.edu/ml/datasets/Polish+companies+bankruptcy+data"
                }
    }
with open(r"E:\WorldQuant\project_5\data\Poland_bankruptcy_modified_test.json","w") as file:
    json.dump(Poland_bankruptcy_modified_test,file,ensure_ascii=False,indent=4)

In [18]:
dict_data_2=[]
import csv
        
with open(r"E:\WorldQuant\project_5\data\Poland_bankruptcy_modified_test.csv", newline='') as f:
    reader = csv.DictReader(f)
    
    for row in reader:
        
        myfixedrow = {k: (None if v=="" else (int(v)if k=="company_id" else float(v))) for k,v in row.items()}
        
        dict_data_2.append(myfixedrow)

In [19]:
json_file = Path(r"E:\WorldQuant\project_5\data\Poland_bankruptcy_modified_test.json")
json_data = json.loads(json_file.read_text())

data = [index for index in dict_data_2]
json_data["data"] = data

json_file.write_text(json.dumps(json_data))

673715

In [20]:
def wrangle_2(filename):
    
    # Open compressed file, load into dictionary
    with open(filename,"r") as f:
        data = json.load(f)
    # Load dictionary into DataFrame, set index
    df=pd.DataFrame.from_dict(data["data"])
    
    return df

In [24]:
df2 = wrangle_2(r"E:\WorldQuant\project_5\data\Poland_bankruptcy_modified_test.json")
print(df2.shape)
df2.head()

(526, 65)


,company_id,Attr_1,Attr_2,Attr_3,Attr_4,Attr_5,Attr_6,Attr_7,Attr_8,Attr_9,...,Attr_55,Attr_56,Attr_57,Attr_58,Attr_59,Attr_60,Attr_61,Attr_62,Attr_63,Attr_64
0,9,0.238950,0.554730,0.40697,1.76090,-22.9070,0.00000,0.297140,0.80268,2.33170,...,3315.200,0.140380,0.536630,0.87292,0.044584,4.4527,7.1847,83.727,4.3594,40.09700
1,28,0.150790,0.584200,0.23091,1.44850,85.8890,0.00000,0.185430,0.71174,0.87949,...,6854.900,0.215930,0.362660,0.79215,0.000000,12.0940,1.3080,213.660,1.7083,3.45900
2,40,0.013141,0.386260,0.38047,2.24300,44.2000,0.00000,0.013141,1.58890,0.88321,...,3220.000,-0.132720,0.021412,0.98720,0.000000,3.3917,4.1778,126.500,2.8855,2.81780
3,81,-0.010051,0.545700,-0.00150,0.99725,-31.1180,0.12076,-0.010051,0.83252,3.35550,...,-2.081,-0.008543,-0.022123,1.00300,0.000000,11.8690,13.5820,59.359,6.1490,7.36170
4,83,0.033372,0.089748,0.10903,2.27890,-2.6961,0.28508,0.033372,10.14200,0.28484,...,2097.100,-0.318780,0.036663,0.92596,0.000000,2.5771,4.0406,109.240,3.3413,0.35351


In [22]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 526 entries, 0 to 525
Data columns (total 65 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   company_id  526 non-null    int64  
 1   Attr_1      526 non-null    float64
 2   Attr_2      526 non-null    float64
 3   Attr_3      526 non-null    float64
 4   Attr_4      526 non-null    float64
 5   Attr_5      523 non-null    float64
 6   Attr_6      526 non-null    float64
 7   Attr_7      526 non-null    float64
 8   Attr_8      526 non-null    float64
 9   Attr_9      526 non-null    float64
 10  Attr_10     526 non-null    float64
 11  Attr_11     526 non-null    float64
 12  Attr_12     526 non-null    float64
 13  Attr_13     524 non-null    float64
 14  Attr_14     526 non-null    float64
 15  Attr_15     526 non-null    float64
 16  Attr_16     526 non-null    float64
 17  Attr_17     526 non-null    float64
 18  Attr_18     526 non-null    float64
 19  Attr_19     524 non-null    f

In [23]:
# gzip compress file
with open(r"E:\WorldQuant\project_5\data\Poland_bankruptcy_modified_test.json",'rb') as f_input:
    with gzip.open(r"E:\WorldQuant\project_5\data\Poland_bankruptcy_modified_test.json.gz",'wb') as f_output:
        shutil.copyfileobj(f_input,f_output)
